In [140]:
import json
import pandas as pd
import numpy as np
import re
import os
import sqlalchemy

from sqlalchemy import create_engine
import psycopg2

from config import db_password

In [141]:
# Credentials
DB_HOST = "127.0.0.1"
DB_PORT = "5432"
DB_NAME = "DiabeticDB"
DB_USER = "postgres"
DB_PASS = db_password
CSV_FILE_PATH = "/Users/anusuyapoonja/Bootcamp_Analysis/Modules/Project/Dataset/diabetic_data_initial.csv"

In [142]:
conn = psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST, port=DB_PORT)
print("Database opened successfully")

Database opened successfully


In [143]:
# Create a cursor
cur = conn.cursor()
# Drop the diabetes_raw_data if exists before creating one
cur.execute("DROP TABLE IF EXISTS medicines_info");
cur.execute("DROP TABLE IF EXISTS admission_info");
cur.execute("DROP TABLE IF EXISTS diagnosis_info");
cur.execute("DROP TABLE IF EXISTS patient_info");
cur.execute("DROP TABLE IF EXISTS diabetes_clean_data");
cur.execute("DROP TABLE IF EXISTS diabetes_raw_data");
conn.commit()

In [144]:
# Create diabetes_raw_data table
cur = conn.cursor()
cur.execute('''CREATE TABLE diabetes_raw_data (
    encounter_id int NOT NULL,
    patient_nbr	int NOT NULL,
    race varchar(20),
    gender varchar(20) NOT NULL,
    age varchar(10) NOT NULL,
    weight varchar(10),
    admission_type_id int NOT NULL,
    discharge_disposition_id int NOT NULL,
    admission_source_id int NOT NULL,
    time_in_hospital int NOT NULL,
    payer_code varchar(10),
    medical_specialty varchar(40),
    num_lab_procedures  int NOT NULL,
    num_procedures  int NOT NULL,
    num_medications  int NOT NULL,
    number_outpatient  int NOT NULL,
    number_emergency  int NOT NULL,
    number_inpatient  int NOT NULL,
    diag_1  varchar(10),
    diag_2 varchar(10),
    diag_3 varchar(10),
    number_diagnoses  int NOT NULL,
    max_glu_serum  varchar(10),
    A1Cresult  varchar(10),
    metformin  varchar(10),
    repaglinide  varchar(10),
    nateglinide  varchar(10),
    chlorpropamide  varchar(10),
    glimepiride  varchar(10),
    acetohexamide  varchar(10),
    glipizide  varchar(10),
    glyburide  varchar(10),
    tolbutamide  varchar(10),
    pioglitazone  varchar(10),
    rosiglitazone  varchar(10),
    acarbose  varchar(10),
    miglitol  varchar(10),
    troglitazone  varchar(10),
    tolazamide  varchar(10),
    examide  varchar(10),
    citoglipton  varchar(10),
    insulin  varchar(10),
    "glyburide-metformin"  varchar(10),
    "glipizide-metformin"  varchar(10),
    "glimepiride-pioglitazone"  varchar(10),
    "metformin-rosiglitazone"  varchar(10),
    "metformin-pioglitazone"  varchar(10),
    change  varchar(10),
    diabetesMed  varchar(10),
    readmitted  varchar(10),
    PRIMARY KEY (encounter_id),
    UNIQUE (encounter_id));''')
print("Table 'diabetes_raw_data' successfully created")

Table 'diabetes_raw_data' successfully created


In [145]:
# Copy the conetnts from the diabetic_data_initial.csv and write it to 'diabetes_raw_data' table
cur = conn.cursor()
cur.execute('''COPY diabetes_raw_data
    FROM '/Users/anusuyapoonja/Bootcamp_Analysis/Modules/Project/Dataset/diabetic_data_initial.csv'
    CSV HEADER DELIMITER ',';''')

In [113]:
# Create the connection to the PostgreSQL database
db_string = f"postgres://{DB_USER}:{db_password}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_string)
conn.commit()

In [114]:
# Read the raw data from the postgres into dataframe
diabetes_raw_data_df = pd.read_sql_table("diabetes_raw_data", engine)
diabetes_raw_data_df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesmed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [115]:
# Clean the raw data in the dataframe diabetes_raw_data_df

diabetes_raw_data_df.drop(['weight', 'payer_code'],axis=1, inplace=True);
diabetes_clean_data_df = diabetes_raw_data_df


In [116]:
# rename the encounter_id name to e_id
#diabetes_clean_data_df = diabetes_clean_data_df.rename(columns={"encounter_id": "e_id"})
diabetes_clean_data_df

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesmed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,?,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,?,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,?,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,?,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),1,3,7,3,?,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),1,4,5,5,?,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),1,1,7,1,?,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),2,3,7,10,Surgery-General,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [117]:
# Create a cursor
cur = conn.cursor()
# Drop the diabetes_cleaned_data if exists before creating one
cur.execute("DROP TABLE IF EXISTS diabetes_clean_data");
conn.commit()

In [118]:
# Write the cleaned data fro diabetes_clean_data_df into postgres diabetes_clean_data table 
diabetes_clean_data_df.to_sql(name='diabetes_clean_data', con=engine, if_exists='replace', index=False,
            dtype={'encounter_id': sqlalchemy.types.INTEGER(),
                   'patient_nbr' : sqlalchemy.types.INTEGER(),
                   'race' : sqlalchemy.types.VARCHAR(length=20),
                   'gender' : sqlalchemy.types.VARCHAR(length=20),
                   'age' : sqlalchemy.types.VARCHAR(length=10),
    'admission_type_id' : sqlalchemy.types.INTEGER(),
    'discharge_disposition_id' : sqlalchemy.types.INTEGER(),
    'admission_source_id' : sqlalchemy.types.INTEGER(),
    'time_in_hospital' : sqlalchemy.types.INTEGER(),
    'num_lab_procedures' :  sqlalchemy.types.INTEGER(),
    'num_procedures' :  sqlalchemy.types.INTEGER(),
    'num_medications' :  sqlalchemy.types.INTEGER(),
    'number_outpatient' :  sqlalchemy.types.INTEGER(),
    'number_emergency' :  sqlalchemy.types.INTEGER(),
    'number_inpatient' :  sqlalchemy.types.INTEGER(),
    'diag_1' :  sqlalchemy.types.VARCHAR(length=10),
    'diag_2' : sqlalchemy.types.VARCHAR(length=10),
    'diag_3' : sqlalchemy.types.VARCHAR(length=10),
    'number_diagnoses' :  sqlalchemy.types.INTEGER(),
    'max_glu_serum' :  sqlalchemy.types.VARCHAR(length=10),
    'A1Cresult' :  sqlalchemy.types.VARCHAR(length=10),
    'metformin' :  sqlalchemy.types.VARCHAR(length=10),
    'repaglinide' :  sqlalchemy.types.VARCHAR(length=10),
    'nateglinide' :  sqlalchemy.types.VARCHAR(length=10),
    'chlorpropamide' :  sqlalchemy.types.VARCHAR(length=10),
    'glimepiride' :  sqlalchemy.types.VARCHAR(length=10),
    'acetohexamide' :  sqlalchemy.types.VARCHAR(length=10),
    'glipizide' :  sqlalchemy.types.VARCHAR(length=10),
    'glyburide' :  sqlalchemy.types.VARCHAR(length=10),
    'tolbutamide' :  sqlalchemy.types.VARCHAR(length=10),
    'pioglitazone' :  sqlalchemy.types.VARCHAR(length=10),
    'rosiglitazone' :  sqlalchemy.types.VARCHAR(length=10),
    'acarbose' :  sqlalchemy.types.VARCHAR(length=10),
    'miglitol' :  sqlalchemy.types.VARCHAR(length=10),
    'troglitazone' :  sqlalchemy.types.VARCHAR(length=10),
    'tolazamide' :  sqlalchemy.types.VARCHAR(length=10),
    'examide' :  sqlalchemy.types.VARCHAR(length=10),
    'citoglipton' :  sqlalchemy.types.VARCHAR(length=10),
    'insulin' :  sqlalchemy.types.VARCHAR(length=10),
    'glyburide-metformin' :  sqlalchemy.types.VARCHAR(length=10),
    'glipizide-metformin' :  sqlalchemy.types.VARCHAR(length=10),
    'metformin-rosiglitazone' :  sqlalchemy.types.VARCHAR(length=10),
    'metformin-pioglitazone' :  sqlalchemy.types.VARCHAR(length=10),
    'change' : sqlalchemy.types.VARCHAR(length=10),
    'diabetesMed' :  sqlalchemy.types.VARCHAR(length=10),
    'readmitted_recoded' :  sqlalchemy.types.VARCHAR(length=10),
    'medical_specialty_recoded' : sqlalchemy.types.VARCHAR(length=10)})
print("Table 'diabetes_clean_data' successfully created")

Table 'diabetes_clean_data' successfully created


In [119]:
# Setting the encounter_id as 'PRIMARY KEY'
with engine.connect() as con:
    con.execute('ALTER TABLE diabetes_clean_data ADD PRIMARY KEY (encounter_id);')
    conn.commit()

In [120]:
# Drop the 'patient_info' if exists before creating one
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS patient_info");

In [121]:
# Create 'patient_info' table
cur = conn.cursor()
cur.execute('''CREATE TABLE patient_info (
    patient_nbr	int NOT NULL,
    race varchar(20),
    gender varchar(20) NOT NULL,
    age varchar(10) NOT NULL,
    PRIMARY KEY (patient_nbr),
    UNIQUE (patient_nbr));''')
print("Table 'patient_info' created successfully")
conn.commit()

Table 'patient_info' created successfully


In [122]:
# Add contents from 'diabetes_cleaned_data' into 'patient_info'
cur = conn.cursor()
cur.execute('''INSERT INTO patient_info (patient_nbr, race, gender, age)
    SELECT DISTINCT patient_nbr, race, gender, age
    FROM diabetes_clean_data
    ON CONFLICT (patient_nbr) DO NOTHING;''')
conn.commit()

In [123]:
# Drop the 'admission_info' if exists before creating one
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS admission_info");

In [124]:
# Create 'admission_info' table
cur = conn.cursor()
cur.execute('''CREATE TABLE admission_info (
    encounter_id int NOT NULL,
    patient_nbr	int NOT NULL,
    admission_type_id int NOT NULL,
    discharge_disposition_id int NOT NULL,
    admission_source_id int NOT NULL,
    time_in_hospital int NOT NULL,
    medical_specialty_recoded varchar(40),
    FOREIGN KEY (encounter_id) REFERENCES diabetes_clean_data (encounter_id),
    PRIMARY KEY (encounter_id));''')
print("Table 'admission_info' successfully created")
conn.commit()   

Table 'admission_info' successfully created


In [125]:
# Add contents from 'diabetes_cleaned_data' into 'admission_info'
cur = conn.cursor()
cur.execute('''INSERT INTO admission_info (encounter_id, patient_nbr, admission_type_id, discharge_disposition_id,
                   admission_source_id, time_in_hospital, medical_specialty_recoded)
            SELECT DISTINCT encounter_id, patient_nbr, admission_type_id, discharge_disposition_id,
                      admission_source_id, time_in_hospital, medical_specialty_recoded
            FROM diabetes_clean_data
            ON CONFLICT (encounter_id) DO NOTHING;''')
conn.commit()

In [126]:
# Drop the 'diagnosis_info' if exists before creating one
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS diagnosis_info");

In [127]:
# Create 'diagnosis_info' table
cur = conn.cursor()
cur.execute('''CREATE TABLE diagnosis_info (
    encounter_id int NOT NULL,
    num_lab_procedures  int NOT NULL,
    num_procedures  int NOT NULL,
    num_medications  int NOT NULL,
    number_outpatient  int NOT NULL,
    number_emergency  int NOT NULL,
    number_inpatient  int NOT NULL,
    diag_1  varchar(10),
    diag_2 varchar(10),
    diag_3 varchar(10),
    number_diagnoses  int NOT NULL,
    max_glu_serum  varchar(10),
    A1Cresult  varchar(10),
    change  varchar(10),
    diabetesMed  varchar(10),
    readmitted_recoded  varchar(10),
    FOREIGN KEY (encounter_id) REFERENCES diabetes_clean_data (encounter_id),
    PRIMARY KEY (encounter_id));''')

print("Table 'diagnosis_info' successfully")
conn.commit()

Table 'diagnosis_info' successfully


In [128]:
# Add contents from 'diabetes_cleaned_data' into 'diagnosis_info'
cur = conn.cursor()
cur.execute('''INSERT INTO diagnosis_info (encounter_id, num_lab_procedures, num_procedures, num_medications,
                        number_outpatient, number_emergency, number_inpatient, diag_1, diag_2, diag_3,
                        number_diagnoses, max_glu_serum, A1Cresult, change, diabetesMed, readmitted_recoded)
    SELECT DISTINCT encounter_id, num_lab_procedures, num_procedures, num_medications, number_outpatient,
                    number_emergency, number_inpatient, diag_1, diag_2, diag_3, number_diagnoses,
                    max_glu_serum, A1Cresult, change, diabetesMed, readmitted_recoded
    FROM diabetes_clean_data
    ON CONFLICT (encounter_id) DO NOTHING;''')
conn.commit()

In [129]:
# Drop the 'medicines_info' if exists before creating one
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS medicines_info");

In [130]:
# Create 'medicines_info' table
cur = conn.cursor()
cur.execute('''CREATE TABLE medicines_info (
    encounter_id int NOT NULL,
    metformin  varchar(10),
    repaglinide  varchar(10),
    nateglinide  varchar(10),
    chlorpropamide  varchar(10),
    glimepiride  varchar(10),
    acetohexamide  varchar(10),
    glipizide  varchar(10),
    glyburide  varchar(10),
    tolbutamide  varchar(10),
    pioglitazone  varchar(10),
    rosiglitazone  varchar(10),
    acarbose  varchar(10),
    miglitol  varchar(10),
    troglitazone  varchar(10),
    tolazamide  varchar(10),
    examide  varchar(10),
    citoglipton  varchar(10),
    insulin  varchar(10),
    "glyburide-metformin"  varchar(10),
    "glipizide-metformin"  varchar(10),
    "glimepiride-pioglitazone"  varchar(10),
    "metformin-rosiglitazone"  varchar(10),
    "metformin-pioglitazone"  varchar(10),
    FOREIGN KEY (encounter_id) REFERENCES diabetes_clean_data (encounter_id),
    PRIMARY KEY (encounter_id));''')

print("Table medicines_info successfully")
conn.commit()

Table medicines_info successfully


In [131]:
# Add contents from 'diabetes_cleaned_data' into 'medicines_info'
cur = conn.cursor()
cur.execute('''INSERT INTO medicines_info (encounter_id, metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide,
                      glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone,
                      tolazamide, examide, citoglipton, insulin, "glyburide-metformin", "glipizide-metformin",
                     "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone")
    SELECT DISTINCT encounter_id, metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide,
                glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone,
                tolazamide, examide, citoglipton, insulin, "glyburide-metformin", "glipizide-metformin",
                "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"
    FROM diabetes_clean_data
    ON CONFLICT (encounter_id) DO NOTHING;''')
conn.commit()

In [132]:
conn.close()